# Importação das bibliotecas

Instalação de bibliotecas utilizadas no pré processamento

In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import nltk
import string  
import emoji as emoji
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Abrindo o dataframe 

In [5]:
df = pd.read_excel('../base/base.xlsx', index_col=0)

# Limpeza dos dados

Nesta etapa o foco é limpar da base de dados os dados que não serão utilizados.
Portanto, com essa função são removidos todos os comentários que tenham autoria do BTG Pactual e fotos de terceiros que marcavam a companhia.

In [6]:
# Esta função remove os dados que não serão utilizados para a análise
def clean_data(df):
    # Remove as aspas dos nomes das colunas
    df = df.rename(columns=lambda x: x.replace('\"', ''))
    # Remove comentários feito pelo btg
    dfWithoutAutor = df[df["autor"] != "btgpactual"]
    # Remove comentários nulos
    dfWithoutNull = dfWithoutAutor[dfWithoutAutor["autor"].notnull()]
    # Deixando apenas as interações do tipo comentário e resposta
    dfFinal = dfWithoutNull.loc[(dfWithoutNull['tipoInteracao'] == 'comentário') | (dfWithoutNull['tipoInteracao'] == 'resposta')]
    return dfFinal

#### Caso de teste

In [7]:
# Criando um data frame de teste para validar o pré processamento
data_test = {'id': [1, 2, 3, 4, 5],
             'dataPublicada': ['03/01/2022', '03/04/2022', '03/07/2022', '03/10/2022', '31/12/2022'],
             'autor': ['btgpactual', 'robert87', 'grace46', 'william25', 'john23'],
             'texto': ["Obrigado, 🚀🚀💙", "Bom dia, gostaria de saber como faço pra conseguir falar com um atendente humanizado no chat do btg banking?", 
                       "💙Bom dia, preciso urgente do meu informe de rendimentos do BTLG11 , para declaração do meu IR. Já enviei vários e-mails e até o momento sem resposta. Telefone ninguém atende. Como faço? 🐸", "Pessoal, sempre muito bom mas se eu puder solicitar uma coisa seria legendas 😅", "Sempre é aproveitoso parar o q tá fazendo para ouvi-lo"],
             'sentimento': ['NEUTRAL', 'NEUTRAL', 'NEGATIVE', 'POSITIVE', 'POSITIVE'],
             'tipoInteracao': ["comentário", "comentário", "comentário", "marcação", "comentário"],
             'anomalia': [1, 0, 0, 1, 1],
             'probabilidadeAnomalia': [100, 20, 30, 100, 100],
             'linkPost': ['https://www.instagram.com/p/CapXhB5Lvas/#17935401550894179', 'https://www.instagram.com/p/CaqBiRpsYLm/', 'https://www.instagram.com/p/CaqBiRpsYLm/', 'https://www.instagram.com/p/CapXhB5Lvas/#17926096268158628', 'https://www.instagram.com/p/CaR1j2TuPpD/#17914683365221503'],
             'processado': [0,0,0,1,1],
             'contemHyperlink': [0,0,0,1,1]}
               
df_testing = pd.DataFrame(data_test)

In [8]:
# Testando a função
clean_data_test = df_testing
cleaning_test = clean_data(clean_data_test)

Os resultados foram colocados em células separadas para facilitar a visualização.

In [9]:
# Entrada do caso de teste
clean_data_test

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
0,1,03/01/2022,btgpactual,"Obrigado, 🚀🚀💙",NEUTRAL,comentário,1,100,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0
1,2,03/04/2022,robert87,"Bom dia, gostaria de saber como faço pra conse...",NEUTRAL,comentário,0,20,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
2,3,03/07/2022,grace46,"💙Bom dia, preciso urgente do meu informe de re...",NEGATIVE,comentário,0,30,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
3,4,03/10/2022,william25,"Pessoal, sempre muito bom mas se eu puder soli...",POSITIVE,marcação,1,100,https://www.instagram.com/p/CapXhB5Lvas/#17926...,1,1
4,5,31/12/2022,john23,Sempre é aproveitoso parar o q tá fazendo para...,POSITIVE,comentário,1,100,https://www.instagram.com/p/CaR1j2TuPpD/#17914...,1,1


In [10]:
# Resultado do caso de teste
cleaning_test

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
1,2,03/04/2022,robert87,"Bom dia, gostaria de saber como faço pra conse...",NEUTRAL,comentário,0,20,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
2,3,03/07/2022,grace46,"💙Bom dia, preciso urgente do meu informe de re...",NEGATIVE,comentário,0,30,https://www.instagram.com/p/CaqBiRpsYLm/,0,0
4,5,31/12/2022,john23,Sempre é aproveitoso parar o q tá fazendo para...,POSITIVE,comentário,1,100,https://www.instagram.com/p/CaR1j2TuPpD/#17914...,1,1


#### Aplicação da função no dataframe


Nesta etapa será demonstrado o funcionamento da função na base de dados real 

In [11]:
cleaning_output = clean_data(df)
cleaning_output

,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
id,,,,,,,,,,
4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0
5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0
6,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0
11,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0
18,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0
...,...,...,...,...,...,...,...,...,...,...
3042413,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0
3042713,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0
3043812,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0


## Tratamento de emojis

Visto que os comentários em redes sociais possuem muitos emojis, decidimos transformá-los para palavras com o intuito de utilizá-los também nos modelos

In [12]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [13]:
import emoji

def converter_emoji(text):
    text_converted = emoji.demojize(text)
    return text_converted

In [14]:
pip install pandas numpy emoji

Note: you may need to restart the kernel to use updated packages.


In [15]:
emoji_dict = {
    '😀': 'feliz ',
    '😂': 'risos ',
    '😔': 'triste ',
    '👏': 'palmas ',
    '🥰': 'amável ',
    '💙': 'coração azul ',
    '🙏🏼': 'orando ',
    '✨': 'brilhando ',
    '🤮': 'nojo ',
    '🚀': 'foguete ',
    '👿': 'diabo ',
    '🤢': 'nojo ',
    '🔥': 'fogo ',
    '😡': 'fúria ',
    '😠': 'raiva ',
    '🤣': 'rindo ',
    '😃': 'feliz ',
    '😎': 'curtindo ',
    '😊': 'feliz ',
    '🤩': 'maravilhado ',
    '😋': 'delicioso ',
    '😆': 'risada ',
    '😌': 'calmo ',
    '🤔': 'pensativo ',
    '😷': 'máscara ',
    '🤣': 'muitoRiso ',
    '🥺': 'carinhoso ',
    '👍': 'positivo ',
    '🤯': 'menteExplodida ',
    '😅': 'alívio ',
    '🥰': 'carinhaComCoração ',
    '😓': 'suor ',
    '😑': 'tédio',
    '🤫': 'silêncio',
    '🤝': 'apertoDeMãos',
    '😊': 'sorriso',
    '😍': 'apaixonado',
    '😭': 'choro ',
    '🤗': 'abraço ',
    '🎉': 'festa ',
    '😎': 'descolado ',
    '😱': 'surpresa ',
    '😴': 'sono ',
    '🙌': 'celebração ',
    '🤔': 'pensativo ',
    '😘': 'beijo ',
    '🥳': 'festeiro ',
    '🙄': 'revirarOsOlhos ',
    '😌': 'alívio ',
    '🤫': 'segredo ',
    '😇': 'inocente ',
    '😂': 'muitoEngraçado ',
    '🤔': 'pensando ',
    '😴': 'sono ',
    '🤪': 'loucura ',
    '😢': 'decepcionado_mas_aliviado ',
}

In [16]:
def emoji_to_word(text):
    for emoji_code, emoji_word in emoji_dict.items():
        text = text.replace(emoji_code, emoji_word)
    return text

In [17]:
cleaning_output['texto_e_palavra'] = cleaning_output['texto'].apply(emoji_to_word)
cleaning_output

,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,texto_e_palavra
id,,,,,,,,,,,
4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,Minuto touro de ouro
5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,@ricktolledo Sim
6,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,Queria saber se a Btg banking é a própria btg ...
11,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0,@thaotinhasbfc
18,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,palmas palmas palmas palmas
...,...,...,...,...,...,...,...,...,...,...,...
3042413,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,Excelente explicação
3042713,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,Porque o brg tá diminuído o limite do cart...
3043812,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Atendam o telefone pelo amor de Deus


In [18]:
# Testando a função 
processing_test = cleaning_output['texto']
emoji_test = processing_test.apply(emoji_to_word)

In [19]:
cleaning_output['texto_e_emoji'].head(20)

id
4                                  Minuto touro de ouro
5                                      @ricktolledo Sim
6     Queria saber se a Btg banking é a própria btg ...
11                                       @thaotinhasbfc
18                         palmas palmas palmas palmas 
19    Vamos ajudar o Brasil na principal categoria d...
20    Com tanta conta digital dando baile nestes ban...
22    @hericles.sisnando o Instagram só deixa public...
23     Cadê a de dividendos? decepcionado_mas_aliviado 
25    Estou aqui apenas para mostrar meu apreço à mu...
26                                coração azul foguete 
31    Eita que não existe ninguém pra responder nem ...
32    Eu mandei pergunta no privado e vcs não me res...
33            Como alocar  valor nessa nova atualização
34    @elson.linss eu recomendo o maior banco de inv...
35                                           Disciplina
37      @gwm.investments e @btgpactual muito alinhados!
43                     Qual horário especial 

## Remoção de stop words e 

Nesta etapa as frases são tokenizadas (cada palavra da frase é dividida, sendo 1 palavra = 1 token) e removidas as stop words, palavras que não acrescentam significado semântico relevante (e.g: de, para, ou). Além disso, é gerado uma nova coluna que contém os tratamento citados acima aplicado aos comentários (coluna 'texto'). 

In [20]:
def processing_comments(frase):
    #divide as frases em palavrasa
    palavras = frase.split() 
    
    # Defina as stop words personalizadas aqui
    stop_words = ['banco','é', 'btg', 'brg', 'pactual', 'btgpactual','pq', 'q', 'pra', 'vcs', 'vc', 'i', 'p', 'kkk', 'y', 'of', 'n', 'a', 'à', 'as', 'o', 'os', 'e', 'aos', 'dos', 'das', 'de', 'deles', 'dela', 'deles', 'delas', 'para', 'que', 'em', 'algo', 'algum', 'alguma', 'alguns', 'algumas', 'aqui', 'aquele', 'aquela', 'aqueles', 'aquelas', 'aqui', 'aquilo', 'cá', 'com', 'como', 'cada', 'coisa', 'daquele', 'daquela', 'daquilo', 'daqueles', 'daquelas', 'desse', 'deste', 'dessa', 'desses', 'destes', 'destas', 'ele', 'eles', 'ela', 'elas', 'eu', 'nos', 'nós', 'vocês', 'voces', 'enquanto', 'era',  'está', 'estamos', 'estão', 'estar', 'estará', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse','estivessem', 'estivéssemos', 'estiveste', 'estivestes', 'estou',  'fará', 'farta', 'farto', 'fez', 'fim', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos','foste', 'fostes', 'fui', 'fôssemos', 'há', 'houve', 'hoje', 'isso', 'isto', 'já', 'lá', 'lhe', 'lhes', 'lo', 'logo',  'mas', 'me', 'mesma', 'mesmas', 'mesmo', 'mesmos', 'meu', 'meus',  'minha', 'minhas', 'na', 'no', 'nas', 'nos', 'naquela', 'naquelas', 'naquele', 'naqueles', 'nem', 'nessa', 'nessas', 'nesse', 'nesses', 'nesta', 'nestas', 'neste', 'nestes', 'ninguém', 'nosso', 'nossa', 'nossos', 'nossas', 'num', 'numa', 'outra', 'outras', 'outro', 'outros', 'pela', 'pelo', 'perante', 'pois', 'ponto', 'pontos', 'por', 'porém', 'porque', 'porquê', 'própria', 'próprio', 'próprias', 'próprios', 'qual', 'quando', 'quanto', 'quantos', 'quantas', 'quê', 'quem', 'quer', 'quereis', 'querem', 'queremas', 'quis', 'quisemos', 'quiser', 'quisera', 'quiseram','quiséramos', 'quiserem', 'quisermos', 'quisésseis', 'quiséssemos', 'quiseste', 'quisestes', 'quiseste','quisestes', 'quizer', 'quizeram', 'quizerem', 'quizermos', 'quizesse', 'quizessem', 'quizéssemos', 'são', 'se', 'seja', 'sejam', 'sejamos', 'sem', 'sendo', 'ser', 'será', 'serão', 'será', 'seriam', 'seríamos','serias', 'seríeis', 'sete', 'seu', 'seus', 'sob', 'sobre', 'sois', 'só','somos', 'sou', 'sua', 'suas', 'tal','tá' 'talvez', 'também', 'te', 'tem', 'têm', 'temos', 'tendes', 'tenha', 'tenham', 'tenhamos', 'tenho', 'tens', 'ter', 'terá', 'terão','terá', 'teriam', 'teríamos', 'terias', 'teríeis', 'teu', 'teus', 'teve', 'tivemos', 'tiver', 'tivera','tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tiveste', 'tivestes','tiveste', 'tivestes', 'um', 'uma', 'umas', 'uns']

    #coloca todas as letras em minúsculas (lower) e remove as stop words
    filtered_words = [word.lower() for word in palavras if word.lower() not in stop_words
                        and not word.startswith('@') and word.isalpha()] 
    #tokeniza as palavras e organiza cada comentário em uma lista de palavras
    tokens = word_tokenize(" ".join(filtered_words)) 
   
    return tokens

#### Caso de teste

In [21]:
processing_test = cleaning_output['texto_e_palavra']
processing_output = processing_test.apply(processing_comments)

In [22]:
# Input da função
processing_test

id
4                                       Minuto touro de ouro
5                                           @ricktolledo Sim
6          Queria saber se a Btg banking é a própria btg ...
11                                            @thaotinhasbfc
18                              palmas palmas palmas palmas 
                                 ...                        
3042413                                 Excelente explicação
3042713    Porque  o brg tá  diminuído  o limite  do cart...
3043812                 Atendam o telefone pelo amor de Deus
3044213    Estou muito grato por todo o esforço que você ...
3045012    @btgpactual pq o morning call não está mais ap...
Name: texto_e_palavra, Length: 6356, dtype: object

In [23]:
# Resultado da função
processing_output

id
4                                      [minuto, touro, ouro]
5                                                      [sim]
6                                   [queria, saber, banking]
11                                                        []
18                          [palmas, palmas, palmas, palmas]
                                 ...                        
3042413                              [excelente, explicação]
3042713    [tá, diminuído, limite, do, cartão, sempre, pa...
3043812                      [atendam, telefone, amor, deus]
3044213    [muito, grato, todo, esforço, você, rendeu, in...
3045012    [morning, call, não, mais, aparecendo, spotify...
Name: texto_e_palavra, Length: 6356, dtype: object

#### Aplicação da função

Nesta etapa será demonstrado o funcionamento da função na base de dados real

In [24]:
cleaning_output['texto_tokenizado'] = cleaning_output['texto_e_palavra'].apply(processing_comments)
cleaning_output

,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,texto_e_palavra,texto_tokenizado
id,,,,,,,,,,,,
4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,Minuto touro de ouro,"[minuto, touro, ouro]"
5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,@ricktolledo Sim,[sim]
6,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,Queria saber se a Btg banking é a própria btg ...,"[queria, saber, banking]"
11,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0,@thaotinhasbfc,[]
18,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,palmas palmas palmas palmas,"[palmas, palmas, palmas, palmas]"
...,...,...,...,...,...,...,...,...,...,...,...,...
3042413,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,Excelente explicação,"[excelente, explicação]"
3042713,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,Porque o brg tá diminuído o limite do cart...,"[tá, diminuído, limite, do, cartão, sempre, pa..."
3043812,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Atendam o telefone pelo amor de Deus,"[atendam, telefone, amor, deus]"


In [25]:
# Exportando a base 
cleaning_output.to_csv('..\\output\\pre_processamento.csv',encoding='utf-8', index=False, header=True, )

### Etapa 2 

Nesta nova etapa iremos abrir a base pré processada e remover todas as colunas que estão vazias 

In [26]:
# Abrindo a base já pré-processada 
df = pd.read_csv('../output/pre_processamento.csv')

In [27]:
# Substituindo todos os elementos vazios da coluna texto_tokenizado por NaN
df['texto_tokenizado'].replace('[]', np.nan, inplace=True)
df

,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,texto_e_palavra,texto_tokenizado
0,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,Minuto touro de ouro,"['minuto', 'touro', 'ouro']"
1,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,@ricktolledo Sim,['sim']
2,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,Queria saber se a Btg banking é a própria btg ...,"['queria', 'saber', 'banking']"
3,"""2022-03-03 18:56:00""",claudiofalavinha,@thaotinhasbfc,NEUTRAL,comentário,0,41,https://www.instagram.com/p/CapXhB5Lvas/#17942...,0,0,@thaotinhasbfc,NaN
4,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,palmas palmas palmas palmas,"['palmas', 'palmas', 'palmas', 'palmas']"
...,...,...,...,...,...,...,...,...,...,...,...,...
6351,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,Excelente explicação,"['excelente', 'explicação']"
6352,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,Porque o brg tá diminuído o limite do cart...,"['tá', 'diminuído', 'limite', 'do', 'cartão', ..."
6353,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Atendam o telefone pelo amor de Deus,"['atendam', 'telefone', 'amor', 'deus']"
6354,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Estou muito grato por todo o esforço que você ...,"['muito', 'grato', 'todo', 'esforço', 'você', ..."


In [28]:
# Removendo todos os NaN
df_processed = df.dropna()
df_processed

,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink,texto_e_palavra,texto_tokenizado
0,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0,Minuto touro de ouro,"['minuto', 'touro', 'ouro']"
1,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0,@ricktolledo Sim,['sim']
2,"""2022-03-03 20:54:00""",ricktolledo,Queria saber se a Btg banking é a própria btg ...,POSITIVE,comentário,0,20,https://www.instagram.com/p/CapXhB5Lvas/#17935...,0,0,Queria saber se a Btg banking é a própria btg ...,"['queria', 'saber', 'banking']"
4,"""2022-03-03 16:11:00""",paularodrigues.invest,👏👏👏👏,NEUTRAL,comentário,0,15,https://www.instagram.com/p/CaptWrtOT8U/#17943...,0,0,palmas palmas palmas palmas,"['palmas', 'palmas', 'palmas', 'palmas']"
5,"""2022-03-03 15:35:00""",rafael_miller,Vamos ajudar o Brasil na principal categoria d...,POSITIVE,comentário,0,41,https://www.instagram.com/p/CapoqJ3Ne7a/#17972...,0,0,Vamos ajudar o Brasil na principal categoria d...,"['vamos', 'ajudar', 'brasil', 'principal', 'ca..."
...,...,...,...,...,...,...,...,...,...,...,...,...
6351,"""2022-11-30 07:48:00""",perspectiveinvestimentos,Excelente explicação,POSITIVE,comentário,0,15,https://www.instagram.com/p/ClgWR-Ov2H1/#18001...,0,0,Excelente explicação,"['excelente', 'explicação']"
6352,"""2022-11-30 08:42:00""",marlenenelso,Porque o brg tá diminuído o limite do cart...,NEGATIVE,comentário,1,53,https://www.instagram.com/p/ClgWR-Ov2H1/#17878...,0,0,Porque o brg tá diminuído o limite do cart...,"['tá', 'diminuído', 'limite', 'do', 'cartão', ..."
6353,"""2022-11-30 11:27:00""",eduardocolares,Atendam o telefone pelo amor de Deus,POSITIVE,comentário,0,28,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Atendam o telefone pelo amor de Deus,"['atendam', 'telefone', 'amor', 'deus']"
6354,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0,Estou muito grato por todo o esforço que você ...,"['muito', 'grato', 'todo', 'esforço', 'você', ..."
